# Hackathon part 2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sidetable
import warnings
import usaddress

In [2]:
fff = pd.read_csv('/Users/Nina/Downloads/fff.csv')


In [3]:
fff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34092783 entries, 0 to 34092782
Data columns (total 10 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   plate_id               object
 1   plate_type             object
 2   registration_state     object
 3   vehicle_body_type      object
 4   violation_description  object
 5   violation_precinct     int64 
 6   violation_county       object
 7   house_number           object
 8   street_name            object
 9   issue_datetime         object
dtypes: int64(1), object(9)
memory usage: 2.5+ GB


In [4]:
fff['plate_type'] = fff['plate_type'].astype('category')
fff['registration_state'] = fff['registration_state'].astype('category')
fff['vehicle_body_type'] = fff['vehicle_body_type'].astype('category')
fff['violation_county'] = fff['violation_county'].astype('category')

# Convert column 9 to datetime
fff['issue_datetime'] = pd.to_datetime(fff['issue_datetime'])

# Convert column 5 to int32
fff['violation_precinct'] = fff['violation_precinct'].astype('int32')

# Display the DataFrame info to verify the changes
print(fff.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34092783 entries, 0 to 34092782
Data columns (total 10 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   plate_id               object        
 1   plate_type             category      
 2   registration_state     category      
 3   vehicle_body_type      category      
 4   violation_description  object        
 5   violation_precinct     int32         
 6   violation_county       category      
 7   house_number           object        
 8   street_name            object        
 9   issue_datetime         datetime64[ns]
dtypes: category(4), datetime64[ns](1), int32(1), object(4)
memory usage: 1.6+ GB
None


## vehicle_body_type cleaning(Sergey)

In [5]:
fff['vehicle_body_type'].nunique()

3841

In [6]:
 
remap_vehicle_types = {"SUBN" : 'Suburban', 
                       "4DSD" : 'Sedan', 
                       "VAN": 'Van', 
                       "DELV" : 'Delivery Truck', 
                       "SDN" : 'Sedan', 
                       "2DSD": 'Sedan', 
                       "PICK" : 'Pickup', 
                       "REFG" : 'Refrigerator Trailer', 
                       "TRAC": 'Tractor', 
                       "TAXI" : 'Taxi', 
                       "UTIL" : 'Utility', 
                       "4 DR": '4 Door', 
                       "CONV" : 'Convertible', 
                       "BUS" : 'Bus', 
                       "4D": '4 Door', 
                       "WAGO" : 'Wagon', 
                       "TK" : 'Truck', 
                       "MCY": 'Motorcycle',
                       "TRLR" : 'Trailer', 
                       "SW" : 'Wagon', 
                       "P-U": 'Pickup',
                       "4S" : 'Sedan', 
                       "FLAT": 'Flat Bed Truck',
                       "T/CR": 'Tractor Crane',
                       "TRAI": 'Trailer'}

In [7]:
fff['vehicle_body_type'] = fff['vehicle_body_type'].map(remap_vehicle_types).fillna('Others')

In [8]:
fff['vehicle_body_type'].value_counts()

vehicle_body_type
Sedan                   11744653
Suburban                10871161
Van                      5594714
Delivery Truck           2885806
Pickup                    988192
Refrigerator Trailer      321957
Others                    288800
Tractor                   275043
Utility                   273099
Taxi                      150301
Trailer                   146386
Bus                       144502
Convertible               141773
Motorcycle                 96151
Truck                      74443
Flat Bed Truck             51850
Tractor Crane              43436
Wagon                        516
Name: count, dtype: int64

## Street names cleaning

In [9]:
fff.street_name.nunique()

235604

In [10]:
#most popular streets
fff.street_name.value_counts().head(30)

street_name
Broadway            836357
3rd Ave             628769
5th Ave             403012
Madison Ave         384190
Lexington Ave       326870
2nd Ave             304693
1st Ave             286894
7th Ave             256528
Queens Blvd         235521
Amsterdam Ave       226250
8th Ave             226167
6th Ave             214954
Jamaica Ave         186588
Columbus Ave        166624
Park Ave            165237
37th Ave            154562
Coney Island Ave    148448
Roosevelt Ave       143246
White Plains Rd     136623
Nostrand Ave        123219
9th Ave             120383
Main St             114245
Fulton St           111524
Myrtle Ave          109893
Jerome Ave          106472
Atlantic Ave        105445
Flatbush Ave        105016
Northern Blvd        99696
St Nicholas Ave      99463
Steinway St          98049
Name: count, dtype: int64

In [11]:
fff[fff['street_name'].str.contains('Broadwa')].street_name.value_counts()

street_name
Broadway               836357
W Broadway              42007
E Broadway              28995
E Broadway.              6272
Broadway / 31st Lot      6245
Old Broadway             1642
Broadway Ter              383
Broadway Aly              261
Name: count, dtype: int64

Issue: '.'

In [12]:
fff[fff['street_name'].str.contains('3rd')].street_name.unique()

array(['73rd Pl', '83rd St', '63rd Dr', '63rd Rd', '43rd Ave', '103rd St',
       '73rd St', '33rd St', '23rd St', '43rd St', '53rd Ave', '23rd Ave',
       '63rd St', '23rd Rd', '33rd Ave', '63rd Ave', '93rd St', '33rd Rd',
       'E 63rd St', 'W 73rd St', 'W 63rd St', '3rd Ave', '123rd St',
       '73rd Rd', 'W 143rd St', 'E 93rd St', 'W 93rd St', 'W 183rd St',
       '133rd Ave', 'W 53rd St', 'E 83rd St', 'E 73rd St', 'E 53rd St',
       'E 153rd St', 'W 83rd St', 'W 103rd St', 'W 173rd St',
       'W 163rd St', 'W 123rd St', 'E 123rd St', '163rd St',
       'Brighton 3rd St', 'E 173rd St', 'E 163rd St', '243rd St',
       '193rd St', 'W 3rd St', '173rd St', '73rd Ave', '3rd Pl',
       'E 33rd St', 'E 3rd St', 'W 33rd St', 'E 23rd St', 'W 43rd St',
       'E 43rd St', 'E 183rd St', 'W 233rd St', 'Bay 23rd St',
       'E 203rd St', 'E 103rd St', 'W 153rd St', 'E 233rd St', 'S 3rd St',
       'N 3rd St', '83rd Ave', '133rd St', '93rd Ave', '223rd St',
       '133rd Rd', '153rd St', '

In [13]:
#see variants of Broadway
fff[fff['street_name'].str.contains('roadwa')].street_name.value_counts()

street_name
Broadway               836357
W Broadway              42007
E Broadway              28995
E Broadway.              6272
Broadway / 31st Lot      6245
Old Broadway             1642
Broadway Ter              383
Broadway Aly              261
Name: count, dtype: int64

Issues: 1 and the same street is mentioned with predirectional(W,E) or without it, dot in the end.

In [14]:
# Remove '.' 
fff['street_name'] = fff['street_name'].str.replace('.', '')

# Remove 'W ' and 'E ' at the beginning
fff['street_name'] = fff['street_name'].str.replace('W ', '')
fff['street_name'] = fff['street_name'].str.replace('E ', '')

fff.street_name.nunique()


216033

We filtered out about 20000 street names in data only with Broadway cleaning!

In [15]:
fff[fff['street_name'].str.contains('Broadwa')].street_name.value_counts()

street_name
Broadway               913631
Broadway / 31st Lot      6245
Old Broadway             1642
Broadway Ter              383
Broadway Aly              261
Name: count, dtype: int64

In [82]:
street = fff.street_name.value_counts().sort_index().reset_index()
street

,street_name,count
0,,44462
1,(is 216) on 174th st,1
2,(sidof) 45th lenox a,1
3,1001st st,1
4,10024th avej,1
...,...,...
172591,zullette,3
172592,zulvett ave,1
172593,zut 38y,1
172594,zvlett ave,1


In [55]:
street.street_name.sort_values().head(60)

0                            
1        (is 216) on 174th st
2        (sidof) 45th lenox a
3                         034
4                     03rd st
5                    048th st
6                     04th st
7                0th 138th st
8                           1
9                       1 ave
10                         10
11                     10 ave
12                        100
13                  1001st st
14               10024th avej
15                       1009
16                    100f of
17                      100ft
18                 100n j ave
19                     100s t
20                      100st
21                   100st st
22                      100th
23                   100th 01
24            100th 127th ave
25            100th 133rd ave
26            100th 134th ave
27             100th 170th st
28              100th 28th st
29              100th 31st rd
30      100th 5251cloverdaleb
31              100th 5th ave
32             100th adams io
33        

In [ ]:
street.street_name.sort_values().sample(60)

In [57]:
street.street_name.sort_values().tail(60)

173703                 zeck ct
173704               zedeg ave
173705               zeger ave
173706              zegreg ave
173707               zeieg ave
173708               zelzer pl
173709               zeneg ave
173710                  zenega
173711               zenion st
173712              zephyr ave
173713               zerag ave
173714                zerc ave
173715                   zerea
173716               zereb ave
173717               zerec ave
173718              zeredg ave
173719                   zereg
173720               zereg ave
173721                  zerega
173722              zerega ave
173723             zereger ave
173724                  zeregu
173725              zeregu ave
173726               zereh ave
173727              zererg ave
173728                zerg ave
173729               zerge ave
173730              zergeg ave
173731               zerrg ave
173732               zeveg ave
173733                 zion st
173734              zivoni ave
173735  

Issues in streets.street_name: make lower case, remove leading and trailing whitespaces, drop '.', '`', 'E', 'W'
replace 'street'> 'st',... Let's clean them.

First try on a street df

In [81]:
# the same with fff
fff['street_name'] = (
    fff['street_name']
    .str.lower()  # Convert txxo lower case
    .str.strip()  # Remove leading and trailing whitespaces
    .str.replace('.|`|,|#|_|/|-', '')  # Remove dots,backticks...
    .str.replace('street', 'st')
    .str.replace('stereet', 'st')
    .str.replace('streeet', 'st')# Replace 'street' with 'st'
    .str.replace('strt', 'st')
    .str.replace('stq', 'st')
    .str.replace('stt', 'st')
    .str.replace('steet', 'st')
    .str.replace('sreet', 'st')
    .str.replace('sret', 'st')
    .str.replace('str', 'st')
    .str.replace('srt', 'st')
    .str.replace('treet', 'st')
    .str.replace('ae', 'ave')
    .str.replace('`', '')  # Remove backticks
    .str.replace(',', '')
    .str.replace('#', '')
    .str.replace('-', '')
    .str.replace('+', '')
    .str.replace('/', '')
    .str.replace('road', 'rd')
    .str.replace('avenue', 'ave')
    .str.replace('avny', 'ave')
    .str.replace('avco', 'ave')
    .str.replace('avwe', 'ave')
    .str.replace('avene', 'ave')
    .str.replace('aven', 'ave')
    .str.replace('ae', 'ave')
    .str.replace('avnue', 'ave')
    .str.replace('abe', 'ave')
    .str.replace('av enue', 'ave')
    .str.replace(' a v', 'ave')
    .str.replace('avon', 'ave')
    .str.replace('a ave', 'ave')
    .str.replace('avewnue', 'ave')
    .str.replace('ase', 'ave')
    .str.replace('aver', 'ave')
    .str.replace('aveuie', 'ave')
    .str.replace('ave ny', 'ave')
    .str.replace('ave0', 'ave')
    .str.replace('aveu3', 'ave')
    .str.replace('ave3', 'ave')
    .str.replace('avw', 'ave')
    .str.replace('avi', 'ave')
    .str.replace('aveq', 'ave')
    .str.replace('aves', 'ave')
    .str.replace('avge', 'ave')
    .str.replace('brdway', 'broadway')
    .str.replace('place', 'pl')
    .str.replace('pk', 'park')
    .str.replace('1 ', '1st ')
    .str.replace('2 ', '2nd ')
    .str.replace('3 ', '3rd ')
    .str.replace('4 ', '4th ')
    .str.replace('5 ', '5th ')
    .str.replace('6 ', '6th ')
    .str.replace('7 ', '7th ')
    .str.replace('8 ', '8th ')
    .str.replace('9 ', '9th ')
    .str.replace('0 ', '0th ')
    .str.replace(' rd', 'rd')
    .str.replace(' nd', 'nd')
    .str.replace('park lot', '')
    .str.replace('parking lot', '')
    .str.replace('parkg', '')
    .str.replace('parkg lot', '')
    .str.replace('crec', 'crescent')
    .str.replace('cresc', 'crescent')
    .str.replace('cress', 'crescent')
    .str.replace('crest', 'crescent')
    .str.replace('crsct', 'crescent')
    .str.replace('creston', 'crescent')
    .str.replace('crecent', 'crescent')
    .str.replace('cresant', 'crescent')
    .str.replace('cresent', 'crescent')
    .str.replace(r'blg\b', 'blvd', regex=True)
    .str.replace(r' bl\b', ' blvd', regex=True)
    .str.replace(r' bd\b', ' blvd', regex=True)
    .str.replace('blvdvd', 'blvd')
    .str.replace(r'\d+ft ', '', regex=True)
    .str.replace('1000th ', '')
    .str.replace(r'\d+th ft ', '', regex=True)
    .str.replace(r'^0+',  '',regex=True)
    .str.replace(r'\d+th co ', '',regex=True)
    .str.replace(r'\d+th eo ', '',regex=True)
    .str.replace(r'\d+th from', '',regex=True)
    .str.replace(r'\d+th cn of', '',regex=True)
    .str.replace(r'\d+th nco', '',regex=True)
    .str.replace('  ', ' ')
    .str.replace(r'\d+th no', '',regex=True)
    .str.replace(r'\d+th  so', '',regex=True)
    .str.replace(r'\d+th of', '',regex=True)
    .str.replace('100th  co', '',regex=True)
    .str.replace('east ', '')
    .str.replace('away', '')
    .str.replace('west', '')
    .str.replace('wof ', '')
    .str.replace('south of ', '')
    .str.replace('north of ', '')
    .str.replace('  ', ' ')
    .str.replace(r' av$', ' ave', regex=True)
    .str.replace(r'^\d+$', '',regex=True)
    .str.replace(r'(\w)(ave)\b', r'\1 \2', regex=True)
    .str.replace(r'(\D)(rd)\b', r'\1 \2', regex=True)
    .str.replace(r'(\w)(str)\b', r'\1 \2', regex=True)
    .str.replace(r'a rd\b', 'ard', regex=True)
    .str.replace(r'o rd\b', 'ord', regex=True)
    .str.replace(r'e rd\b', 'erd', regex=True)
    .str.replace(r'jamaic\b', 'jamaica',regex=True )

    .str.strip()
)


# Display unique street names after transformations
print("Unique street names after transformations:", fff['street_name'].nunique())

Unique street names after transformations: 172596


In [88]:
fff.street_name.value_counts().tail()

street_name
broadway                 988580
3rd ave                  697007
5th ave                  448735
madison ave              411574
lexington ave            353496
                          ...  
shcenck st                    1
fo 10440th coron ave          1
320th w15351                  1
deccatur st                   1
so terminal 6th jfkia         1
Name: count, Length: 172596, dtype: int64

In [89]:
fff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34092783 entries, 0 to 34092782
Data columns (total 10 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   plate_id               object        
 1   plate_type             category      
 2   registration_state     category      
 3   vehicle_body_type      object        
 4   violation_description  object        
 5   violation_precinct     int32         
 6   violation_county       category      
 7   house_number           object        
 8   street_name            object        
 9   issue_datetime         datetime64[ns]
dtypes: category(3), datetime64[ns](1), int32(1), object(5)
memory usage: 1.7+ GB


In [90]:
fff.stb.missing()

,missing,total,percent
house_number,609567,34092783,1.787965
plate_id,0,34092783,0.000000
plate_type,0,34092783,0.000000
registration_state,0,34092783,0.000000
vehicle_body_type,0,34092783,0.000000
violation_description,0,34092783,0.000000
violation_precinct,0,34092783,0.000000
violation_county,0,34092783,0.000000
street_name,0,34092783,0.000000
issue_datetime,0,34092783,0.000000


We have 1.7+GB of data in 10 columns 34092783 rows, 1.79% of house number values are missing.

## Exclude one issue streets

In [91]:
#check number of cases per street
str_count= fff.street_name.value_counts().reset_index()
str_count

,street_name,count
0,broadway,988580
1,3rd ave,697007
2,5th ave,448735
3,madison ave,411574
4,lexington ave,353496
...,...,...
172591,shcenck st,1
172592,fo 10440th coron ave,1
172593,320th w15351,1
172594,deccatur st,1


In [92]:
#find streets which had issues only once
str_count[str_count['count']==1]


,street_name,count
56410,wb sn 7th,1
56411,manaham ave,1
56412,sb io lexington ave,1
56413,flatbush avne,1
56414,desarc dr,1
...,...,...
172591,shcenck st,1
172592,fo 10440th coron ave,1
172593,320th w15351,1
172594,deccatur st,1


In [93]:
# find the share of these streets
116186/len(str_count)

0.6731673966951726

In [94]:
# find streets which had  <100 issues
str_count[str_count['count']< 100]

,street_name,count
5200,burchell ave,99
5201,queen blvd,99
5202,bx park e,99
5203,81st rd,99
5204,fillmor ave,99
...,...,...
172591,shcenck st,1
172592,fo 10440th coron ave,1
172593,320th w15351,1
172594,deccatur st,1


In [95]:
167396/len(str_count)

0.9698718394400797

67% of all street appear in fff only once, 97% of all streets have less than 100 issues in 4 years. Let's find their share in the whole fff.

In [96]:
# Filter out rows where 'street_name' appear only once
threshold = 1
counts = fff['street_name'].value_counts()
desired_values = counts[counts <= threshold].index
one_issue = fff[fff['street_name'].isin(desired_values)]

one_issue.info()

<class 'pandas.core.frame.DataFrame'>
Index: 116186 entries, 10 to 34092778
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   plate_id               116186 non-null  object        
 1   plate_type             116186 non-null  category      
 2   registration_state     116186 non-null  category      
 3   vehicle_body_type      116186 non-null  object        
 4   violation_description  116186 non-null  object        
 5   violation_precinct     116186 non-null  int32         
 6   violation_county       116186 non-null  category      
 7   house_number           45928 non-null   object        
 8   street_name            116186 non-null  object        
 9   issue_datetime         116186 non-null  datetime64[ns]
dtypes: category(3), datetime64[ns](1), int32(1), object(5)
memory usage: 7.0+ MB


In [30]:
len(one_issue)/len(fff)

0.0034297581397212426

So these streets appear in data only once(mostly typos and errors), constitute 0.34% of data but 67.3% of all streets.

In [31]:
one_issue.stb.missing()

,missing,total,percent
house_number,70718,116930,60.478919
plate_id,0,116930,0.000000
plate_type,0,116930,0.000000
registration_state,0,116930,0.000000
vehicle_body_type,0,116930,0.000000
violation_description,0,116930,0.000000
violation_precinct,0,116930,0.000000
violation_county,0,116930,0.000000
street_name,0,116930,0.000000
issue_datetime,0,116930,0.000000


So, in one_issue the majority of street names are typos or errors(invalid street names), and in 60% cases they also don't have house numbers. We may easily drop them.

In [97]:
threshold = 100
counts = fff['street_name'].value_counts()
desired_values = counts[counts <threshold].index
hundred_is = fff[fff['street_name'].isin(desired_values)]

hundred_is.info()

<class 'pandas.core.frame.DataFrame'>
Index: 587705 entries, 5 to 34092779
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   plate_id               587705 non-null  object        
 1   plate_type             587705 non-null  category      
 2   registration_state     587705 non-null  category      
 3   vehicle_body_type      587705 non-null  object        
 4   violation_description  587705 non-null  object        
 5   violation_precinct     587705 non-null  int32         
 6   violation_county       587705 non-null  category      
 7   house_number           305286 non-null  object        
 8   street_name            587705 non-null  object        
 9   issue_datetime         587705 non-null  datetime64[ns]
dtypes: category(3), datetime64[ns](1), int32(1), object(5)
memory usage: 35.3+ MB


In [99]:
len(hundred_is)/len(fff)

0.017238399106344587

In [34]:
hundred_is.stb.missing()

,missing,total,percent
house_number,282974,593112,47.710045
plate_id,0,593112,0.000000
plate_type,0,593112,0.000000
registration_state,0,593112,0.000000
vehicle_body_type,0,593112,0.000000
violation_description,0,593112,0.000000
violation_precinct,0,593112,0.000000
violation_county,0,593112,0.000000
street_name,0,593112,0.000000
issue_datetime,0,593112,0.000000


1,7% of data are streets with less than 100 issues - 97% of all streets.  these street definitely don't need more parking lots.

Let's exclude rows with one_issue streets from our data

In [100]:
threshold = 1
counts = fff['street_name'].value_counts()
desired_values = counts[counts > threshold].index
data = fff[fff['street_name'].isin(desired_values)]
data = data.reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33976597 entries, 0 to 33976596
Data columns (total 10 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   plate_id               object        
 1   plate_type             category      
 2   registration_state     category      
 3   vehicle_body_type      object        
 4   violation_description  object        
 5   violation_precinct     int32         
 6   violation_county       category      
 7   house_number           object        
 8   street_name            object        
 9   issue_datetime         datetime64[ns]
dtypes: category(3), datetime64[ns](1), int32(1), object(5)
memory usage: 1.7+ GB


In [36]:
len(data)/len(fff) 

0.9965702418602788

In [101]:
data.stb.missing()

,missing,total,percent
house_number,539309,33976597,1.587296
plate_id,0,33976597,0.000000
plate_type,0,33976597,0.000000
registration_state,0,33976597,0.000000
vehicle_body_type,0,33976597,0.000000
violation_description,0,33976597,0.000000
violation_precinct,0,33976597,0.000000
violation_county,0,33976597,0.000000
street_name,0,33976597,0.000000
issue_datetime,0,33976597,0.000000


## to csv

In [102]:
file_path = '/Users/Nina/Downloads/data.csv'
data.to_csv(file_path, index=False)